<h1 align="center">Ejemplos de linealización de datos</h1>
<div align="right">David A. Miranda, PhD<br>2022</div>

## 1. Importar librería

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
is_colab = False
try:
    from google.colab import files
    is_colab = True
except:
    pass

## 2. Cargar los datos

Para este ejercicio se utilizarán datos de la Tasa Representativa del Mercado (TRM - peso por dolar) publicados por el Banco de la República de Colombia: [https://www.banrep.gov.co/es/estadisticas/trm](https://www.banrep.gov.co/es/estadisticas/trm).

Si usa Colab, suba el archivo .xlsx. 
Si trabaja local, de ser el caso, actualice la ruta y nombre del arcivho en **excel_data**.

In [ ]:
df = pd.DataFrame([])
if not is_colab:
    excel_data = '../data/TRM2022.xlsx'
    df = pd.read_excel(excel_data, header=8)
if is_colab:
    uploaded = files.upload()
df = pd.read_excel(list(uploaded.keys())[0], header=8)
df

## 3. Analítica de datos

### 3.1. Limpieza y pre-procesamiento de los datos

In [ ]:
t = []
x = []
for index, row in df.iterrows():
    this_date = row['Fecha (dd/mm/aaaa)']
    this_TMR = row['TRM (COP/USD)']
    if not type(this_date) == type(dt.datetime.today()):
        # Descarta los datos donde la columna de fechas no tienen formato de fecha
        continue
    t.append(this_date.timetuple().tm_yday)
    x.append(this_TMR)
t = np.array(t)
x = np.array(x)

### 3.2. Obtención de pendientes e interceptos: linealización

In [ ]:
def linealize(tau=15): # días
    delta_t = []
    delta_x = []
    slopes = []
    intercepts = []

    for idx, i in enumerate(range(tau, len(t), tau)):
        # Pendientes e interceptos de tramos pares
        this_t = t[i-tau:i]
        this_x = x[i-tau:i]
        p = np.r_[np.nan, np.nan]
        if idx%2 == 0:
            p = np.polyfit(this_t, this_x, 1)
        delta_t.append([this_t[0], this_t[-1]])
        delta_x.append([this_x[0], this_x[-1]])
        slopes.append(p[0])
        intercepts.append(p[1])

    for idx in range(1, len(slopes), 2):
        # Empalme de pendientes de tramos pares con líneas que unen los tramos: pendientes e interceptos impares
        if idx+1 >= len(slopes):
            break
        p1 = slopes[idx-1], intercepts[idx-1]
        p2 = slopes[idx+1], intercepts[idx+1]
        t1, t2 = delta_t[idx]
        x1 = p1[0] * t1 + p1[1]
        x2 = p2[0] * t2 + p2[1]
        slopes[idx] = (x2-x1) / (t2-t1)
        intercepts[idx] = x1 - slopes[idx] * t1
        delta_x[idx] = [x1, x2]
    return delta_t, delta_x, slopes, intercepts

### 3.3. Método para graficar datos y lineal por segmentos (linealización)

In [ ]:
def plot_TMR_linealized(tau=15):
    delta_t, delta_x, slopes, intercepts = linealize(tau=tau)
    plt.plot(t, x,'.', label='TRM diario')
    for k in range(len(slopes)):
    # Gráfica de linealización con identificación de puntos de empalme
        p = np.r_[slopes[k], intercepts[k]]
        plt.plot(delta_t[k], np.polyval(p, delta_t[k]), 'k', lw=2)
        if k == 0:
            plt.plot(delta_t[k], np.polyval(p, delta_t[k]), 'k', lw=2, label='Linealización')
        if k%2 == 1:
            plt.plot(delta_t[k], delta_x[k], 'or')
            if k==1:
                plt.plot(delta_t[k], delta_x[k], 'or', label='Empalmes')
                plt.text(delta_t[k][0]-30, delta_x[k][0]+80, '$ %0.1f' % delta_x[k][0], c='r')

### 3.4. Gráfica del modelo linealizado y predicciones

In [ ]:
plt.figure(dpi=300)
plot_TMR_linealized()

plt.legend()
plt.xlabel('Dia del año')
plt.ylabel('TRM COP/USD')
_ = plt.ylim(3500, 5500)

In [ ]:
plt.figure(dpi=300)
plot_TMR_linealized()
delta_t, delta_x, slopes, intercepts = linealize(tau=15)
# Pedicción a 15 días si TMR sigue a la baja
t1 = delta_t[0][0]
t2 = delta_t[0][0] + 15
p = slopes[0], intercepts[0]

t_ext = [t1, t2]
x_ext = np.polyval(p, [t1, t2])
plt.plot(t_ext, x_ext, 'g:', label='Extrapolación con pendiente actual')
plt.plot(t_ext[-1], x_ext[-1], 'go')
plt.text(t_ext[-1]-40, x_ext[-1]-220, '$ %0.1f' % x_ext[-1], c='g')

plt.legend()
plt.xlabel('Dia del año')
plt.ylabel('TRM COP/USD')
_ = plt.ylim(3500, 5500)

In [ ]:
plt.figure(dpi=300)
plot_TMR_linealized()
# Pedicción a diciembre 31 si TMR sigue a la baja

plt.plot(t_ext[-1], x_ext[-1], 'bo')
plt.text(t_ext[-1]-40, x_ext[-1]-220, '$ %0.1f' % x_ext[-1], c='b')

t1 = delta_t[0][0]
t2 = 365
p = slopes[0], intercepts[0]

t_ext = [t1, t2]
x_ext = np.polyval(p, [t1, t2])
plt.plot(t_ext, x_ext, 'g:', label='Extrapolación con pendiente actual')
plt.plot(t_ext[-1], x_ext[-1], 'go')
plt.text(t_ext[-1]-40, x_ext[-1]-120, '$ %0.1f' % x_ext[-1], c='g')

plt.legend()
plt.xlabel('Dia del año')
plt.ylabel('TRM COP/USD')
_ = plt.ylim(3500, 5500)

## 4. Exportar modelo

In [ ]:
slope_df = pd.DataFrame({    
    't0 [day]':[dt[1] for dt in delta_t],
    'TRM(t0) [COP/USD]':[dx[1] for dx in delta_x],
    't1 [day]':[dt[0] for dt in delta_t],
    'TRM(t1) [COP/USD]':[dx[0] for dx in delta_x],
    'slope [COP/USD per day]': list(slopes),
    'intercepts [COP/USD]': list(intercepts)
})
#slope_df.to_excel('../data/2022TRM_LinearizedModel_Slopes_Intercepts.xlsx', index=False)
slope_df

End!